In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import pickle as pkl
import torch.nn.functional as F
import torchvision
from attack import attack, test_model,parse_param


In [2]:
import re
def parse_param(param):
    reg = re.compile("\.\d+\.")
    finded = reg.findall(param)
    if len(finded) == 0:
        pass
    else:
        for f in finded:
            f = f[1:-1]
            param = param.replace(f".{f}.", f"[{f}].")
    return param

In [3]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar10_resnet50()


In [4]:
all_param_names = list()
for name, param in model.named_parameters():
    if not "bn" in name and not "shortcut.1" in name:
        all_param_names.append(name)

In [5]:
all_param_names = all_param_names[:-2]

In [6]:
train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all = load_cifar10()
all_totals = list()
all_totals.append(attack(train_dataloader_all, all_param_names, load_cifar10_resnet50, alpha=0.0001))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 196/196 [01:19<00:00,  2.45it/s]


0.0005741860742866993


100%|██████████| 196/196 [01:18<00:00,  2.50it/s]


0.4307701399612427


...

100%|██████████| 196/196 [01:22<00:00,  2.38it/s]


55.098501591796875


100%|██████████| 196/196 [01:24<00:00,  2.31it/s]


206.471825234375


d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\attack.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  param_totals = np.array(param_totals)
d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\utils.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [7]:
all_totals


[{'conv1.weight': array([[[[0.12940998, 0.19194692, 0.2049628 ],
           [0.09550446, 0.09992224, 0.08056554],
           [0.05099013, 0.11817644, 0.05753839]],
  
          [[0.18637227, 0.25311026, 0.27067748],
           [0.09474684, 0.1192866 , 0.08235797],
           [0.13709514, 0.1118738 , 0.05391484]],
  
          [[0.4053717 , 0.46918887, 0.47307196],
           [0.3258942 , 0.32984805, 0.30694064],
           [0.40497887, 0.33133748, 0.264538  ]]],
  
  
         [[[0.47511494, 0.68679446, 1.1655645 ],
           [0.44857982, 0.48117656, 1.0003198 ],
           [0.6351995 , 0.24398467, 0.41591647]],
  
          [[0.38884762, 0.5312766 , 1.0177957 ],
           [0.4514019 , 0.45789114, 0.92042875],
           [0.6756563 , 0.21781603, 0.4100416 ]],
  
          [[0.3718205 , 0.4994984 , 0.95503557],
           [0.45838165, 0.38496998, 0.89439887],
           [0.7106636 , 0.16929227, 0.4559369 ]]],
  
  
         [[[1.2967029 , 1.3854629 , 1.0212567 ],
           [1.1832013

In [8]:
pkl.dump(all_totals, open("weights/cifar10_resnet50_nolinear_bn.pkl", "wb"))

In [15]:
thre = 0.5
net = load_cifar10_resnet50()
param_remove = dict()
for param in all_param_names:
    param_remove[param] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[param] for param in all_param_names]
    param_weights = [eval("net." + parse_param(param) + ".cpu().detach().numpy()")
                     for param in all_param_names]
    combine = [np.abs(total * weight) for total, weight in zip(totals, param_weights)]
    combine = np.array(combine)
    combine_flatten = np.concatenate([combine_.flatten() for combine_ in combine],axis=0)
    threshold = np.sort(combine_flatten)[::-1][int(len(combine_flatten) * thre)]
    for idx,param in enumerate(all_param_names):
        if param_remove[param] is None:
            param_remove[param] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            param_remove[param] = param_remove[param] | t

C:\Users\Zhang\AppData\Local\Temp\ipykernel_30000\1181857525.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combine = np.array(combine)


In [16]:
temp = 0
all_num = 0
for param in param_remove:
    temp += param_remove[param].sum()
    all_num += param_remove[param].size
    print(param, param_remove[param].mean())

conv1.weight 0.9971064814814815
layer1.0.conv1.weight 0.938720703125
layer1.0.conv2.weight 0.8323838975694444
layer1.0.conv3.weight 0.88739013671875
layer1.0.shortcut.0.weight 0.87091064453125
layer1.1.conv1.weight 0.766357421875
layer1.1.conv2.weight 0.8091634114583334
layer1.1.conv3.weight 0.87457275390625
layer1.2.conv1.weight 0.8057861328125
layer1.2.conv2.weight 0.8533257378472222
layer1.2.conv3.weight 0.84320068359375
layer2.0.conv1.weight 0.919921875
layer2.0.conv2.weight 0.7741970486111112
layer2.0.conv3.weight 0.83294677734375
layer2.0.shortcut.0.weight 0.7821884155273438
layer2.1.conv1.weight 0.66607666015625
layer2.1.conv2.weight 0.79119873046875
layer2.1.conv3.weight 0.8011016845703125
layer2.2.conv1.weight 0.734466552734375
layer2.2.conv2.weight 0.716064453125
layer2.2.conv3.weight 0.740509033203125
layer2.3.conv1.weight 0.796905517578125
layer2.3.conv2.weight 0.7399766710069444
layer2.3.conv3.weight 0.7042999267578125
layer3.0.conv1.weight 0.856658935546875
layer3.0.conv2

In [17]:
temp / all_num

0.5

In [18]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    correct, all = test_model(net, test_dataloader_all)
    print("原始准确率", correct / all)


原始准确率 0.954


In [19]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        try:
            exec("net." + param_ + "[~param_remove[param]] = 0")
        except:
            exec("net." + param_ + "[~param_remove[param],:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("现在准确率", correct / all)


现在准确率 0.8911


In [20]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for param in all_param_names:
        param_ = parse_param(param)
        keep_rate = param_remove[param].sum() / param_remove[param].size
        weight_flatten = eval("net." + param_ + ".cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(len(weight_flatten) * (1 - keep_rate))]
        try:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold] = 0")
        except:
            exec("net." + param_ + "[eval('net.' + param_ + '.cpu().detach().numpy()') < threshold,:] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("对比试验准确率", correct / all)


对比试验准确率 0.1
